# Text summarization using Hugging Face Transformers

The following script demonstartes the usage of the Hugging Face Transformers library to perform text summarization using a pre-trained model. 



In [3]:
import nltk  # Import the Natural Language Toolkit library
nltk.download("punkt")  # Download the 'punkt' tokenizer models from NLTK
from nltk.tokenize import sent_tokenize  # Import the sentence tokenizer from NLTK

from transformers import pipeline, set_seed  # Import the pipeline and set_seed functions from the transformers library


from sumy.parsers.plaintext import PlaintextParser  # Import PlainTextParser from the sumy package
from sumy.nlp.tokenizers import Tokenizer  # Import Tokenizer from the sumy package
from sumy.summarizers.text_rank import TextRankSummarizer  # Import TextRankSummarizer from the sumy package

from datasets import load_dataset  # Import functions to load datasets from the datasets library

import evaluate # Import the evaluate library for model evaluation

import pandas as pd  # Import the pandas library to work with tables


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\elena.jolkver\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Nltk (Natural Language Toolkit) is a comprehensive library for building Python programs to work with human language data. It provides easy-to-use interfaces to over 50 corpora and lexical resources such as WordNet, along with a suite of text processing libraries for classification, tokenization, stemming, tagging, parsing, and more. The 'punkt' tokenizer specifically helps in sentence tokenization, which is the process of dividing a text into a list of its component sentences. 

The transformers library by Huggingface is a state-of-the-art natural language processing library that provides thousands of pretrained models to perform tasks on different modalities such as text, vision, and audio (Hugging Face – The AI Community Building the Future., 2025). The 'pipeline' function simplifies the implementation of complex models for various NLP tasks, including text generation and summarization.

Sumy is a Python library used for text summarization. It offers several algorithms for extracting summaries from text documents. The PlaintextParser is used to read and parse plain text files. TextRankSummarizer is an implementation of the TextRank algorithm, a graph-based summarization technique.


In [ ]:
dataset = load_dataset('ccdv/cnn_dailymail', '3.0.0', split='train[:1]', trust_remote_code = True)

# Access the first article
first_article = dataset[0]
print(first_article['article'])


Generating train split: 16001 examples [02:08, 124.18 examples/s]


KeyboardInterrupt: 

We have selected the CNN/DailyMail dataset (Hermann et al., 2015) for our analysis of summarization models. This dataset includes approximately 300,000 pairs of news articles and their corresponding summaries. These summaries are derived from the bullet points that CNN and the DailyMail add to their articles. A notable feature of this dataset is that the summaries are not mere excerpts but are instead abstractive, generating new sentences that encapsulate the essence of the articles.

For illustrative purposes, we will apply our summarization techniques to a single article. Lengthy articles present a significant challenge to most transformer models due to their typical context size limitation of about 1,000 tokens, which equates to several paragraphs. A common, albeit rudimentary, strategy to manage this constraint is to truncate texts that exceed the model's context size. Although crucial content might reside towards the end of the text, we must navigate this limitation of the model architectures for now, thereby limiting the article to the first 2000 characters:


In [ ]:
sample_text = dataset["train"][1]["article"][:2000]  # Get the first 2000 characters of the second article in the training dataset
sample_text



In the absence of human-generated "true" summaries within the text corpus, we can create a reference point by establishing a simple baseline. This baseline allows us to compare the machine-generated summaries against a consistent benchmark. To achieve this, we extract the first three sentences of the text to serve as our baseline summary.

In [ ]:
# We'll collect the generated summaries of each model in a dictionary
summaries = {}  # Initialize an empty dictionary to store summaries

def three_sentence_summary(text):  # Define a function to generate a three-sentence summary
    return "\n".join(sent_tokenize(text)[:3])  # Return the first three sentences of the text


In [ ]:
summaries["baseline"] = three_sentence_summary(sample_text)  # Generate a baseline summary and store it in the dictionary

The baseline summary, extracted as the first three sentences of the article, already provides a concise encapsulation of the key events: 

In [ ]:
summaries["baseline"]

"(CNN) -- Usain Bolt rounded off the world championships Sunday by claiming his third gold in Moscow as he anchored Jamaica to victory in the men's 4x100m relay.\nThe fastest man in the world charged clear of United States rival Justin Gatlin as the Jamaican quartet of Nesta Carter, Kemar Bailey-Cole, Nickel Ashmeade and Bolt won in 37.36 seconds.\nThe U.S finished second in 37.56 seconds with Canada taking the bronze after Britain were disqualified for a faulty handover."

We now proceed with generating a summary using the classical algorithm provided by the Sumy library. One of the key algorithms provided by Sumy is the TextRankSummarizer, which implements the TextRank algorithm. TextRank is a classical unsupervised algorithm inspired by Google's PageRank method used for ranking web pages. As it doesn’t require pre-labeled data or training, it is versatile option when large annotated datasets aren't available. Its effectiveness in summarizing text, especially when domain-specific training data is scarce, makes it a valuable tool in the toolkit of text summarization techniques. By using the TextRankSummarizer from the Sumy library, we can efficiently generate summaries that aim to capture the essential information and main points of the source documents without needing extensive computational resources or training data.
Here's how TextRank operates in the context of text summarization:

1. Graph-Based Approach: TextRank constructs a graph where sentences are nodes. The edges between sentences are weighted based on the similarity between sentences, typically computed using measures like cosine similarity on word vectors.

2. Sentence Representation: Sentences in the text are represented as nodes in the graph, and the algorithm establishes connections (edges) between these nodes based on semantic similarity; the more similar two sentences are, the stronger the connection.

3. Ranking Sentences: Once the graph is built, the TextRank algorithm applies a ranking process similar to PageRank to score the nodes (sentences). This process iteratively refines the score of each sentence based on its connections and the score of the connected sentences.

4. Extracting Key Sentences: After the ranking process converges, the sentences with the highest scores are extracted as the summary. Typically, the top-ranked sentences are selected to form a coherent and concise summary of the original text.


In [ ]:
parser = PlaintextParser.from_string(sample_text, tokenizer = Tokenizer("english"))  # Use PlainTextParser to parse the sample text
summarizer = TextRankSummarizer()  # Initialize the TextRank summarizer

# Collect the summary sentences in a list
summary_sentences = []  # Initialize an empty list to store summary sentences
for sentence in summarizer(parser.document, 5):  # Generate summary sentences using the TextRank summarizer
    summary_sentences.append(str(sentence))  # Append each summary sentence to the list

# Join the sentences to form a single summary string
summaries["sumy"] = "\n".join(summary_sentences)  # Store the generated summary in the dictionary

# Print the summary (optional)
print(summaries["sumy"])  # Print the summary generated by the TextRank summarizer


(CNN) -- Usain Bolt rounded off the world championships Sunday by claiming his third gold in Moscow as he anchored Jamaica to victory in the men's 4x100m relay.
The fastest man in the world charged clear of United States rival Justin Gatlin as the Jamaican quartet of Nesta Carter, Kemar Bailey-Cole, Nickel Ashmeade and Bolt won in 37.36 seconds.
The 26-year-old Bolt has now collected eight gold medals at world championships, equaling the record held by American trio Carl Lewis, Michael Johnson and Allyson Felix, not to mention the small matter of six Olympic titles.
The relay triumph followed individual successes in the 100 and 200 meters in the Russian capital.
Defending champions, the United States, were initially back in the bronze medal position after losing time on the second handover between Alexandria Anderson and English Gardner, but promoted to silver when France were subsequently disqualified for an illegal handover.


In the final step, we employ three different large language models (LLMs) for summarization: GPT, BART, and a lightweight version of DeepSeek. Among these, GPT-2 and DeepSeek serve as general-purpose LLMs. We prompt these models by appending "TL;DR:" to the article, which is shorthand for "too long; didn’t read" and commonly signals a brief summary. Language models recognize this due to its frequent occurrence in training data, interpreting it as an instruction to summarize. This allows for conditional generation, where the model, given this prefix, produces the subsequent words, forming the summary. In contrast, the specific BART model employed here has been fine-tuned on the CNN/DailyMail dataset for summarization tasks. Consequently, BART acts as a form of positive control in our experiment, given its prior training on the very dataset we are utilizing for demonstration. 

In [ ]:
# GPT2 summary
set_seed(42)  # Set the random seed for reproducibility
pipe = pipeline("text-generation", model="gpt2-xl")  # Initialize a text generation pipeline with the GPT-2 XL model
query = sample_text + "\nTL;DR:\n"  # Create a query for the GPT-2 model
pipe_out = pipe(query, max_new_tokens=1000, clean_up_tokenization_spaces=True)  # Generate text using the GPT-2 model
summaries["gpt2"] = "\n".join(  # Store the generated summary in the dictionary
    sent_tokenize(pipe_out[0]["generated_text"][len(query) :]))  # Tokenize the generated text into sentences
del pipe
del pipe_out
del query

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [1]:
# DeepSeek Summary
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"  # Define the model name for the DeepSeek model
set_seed(42)  # Set the random seed for reproducibility
pipe = pipeline("text-generation", model=model_name)  # Initialize a text generation pipeline with the DeepSeek model
query = sample_text + "\nTL;DR:\n"  # Create a query for the DeepSeek model
pipe_out = pipe(query, max_new_tokens=1000, clean_up_tokenization_spaces=True)  # Generate text using the DeepSeek model
summaries["DeepSeek"] = "\n".join(  # Store the generated summary in the dictionary
    sent_tokenize(pipe_out[0]["generated_text"][len(query) :]))  # Tokenize the generated text into sentences
del pipe
del pipe_out
del query

NameError: name 'set_seed' is not defined

In [ ]:
#BART summary
pipe = pipeline("summarization", model="facebook/bart-large-cnn")  # Initialize a summarization pipeline with the BART model
pipe_out = pipe(sample_text)  # Generate a summary using the BART model
summaries["bart"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))  # Store the generated summary in the dictionary
del pipe
del pipe_out


Device set to use cpu


NameError: name 'query' is not defined

In [ ]:
summaries["DeepSeek"]

"In the men's 4x100 relay, Usain Bolt secured his third gold medal in Moscow, defeating Justin Gatlin, resulting in Jamaica's victory.\nThe U.S. team took second, while Canada and Britain finished third and fourth, respectively.\nThe relay was won by Bolt with Ashmeade providing the baton, and Bolt successfully took control of the baton from Gatlin.\nThe individual performances in the sprint events also contributed to Jamaica's dominance.\n</think>\n\nIn the men's 4x100m relay, Usain Bolt secured his third gold medal in Moscow, defeating Justin Gatlin, resulting in Jamaica's victory.\nThe U.S. team took second, while Canada and Britain finished third and fourth, respectively.\nThe relay was won by Bolt with Ashmeade providing the baton, and Bolt successfully took control of the baton from Gatlin.\nThe individual performances in the sprint events also contributed to Jamaica's dominance."

Comparing summaries

In [ ]:
print("ORIGINAL TEXT")
for sentence in sent_tokenize(sample_text):
    print(sentence)
print("")

print("HUMAN SUMMARY")
print(dataset["train"][1]["highlights"])
print("")

for model_name in summaries:
    print(model_name.upper())
    print(summaries[model_name])
    print("")

ORIGINAL TAXT
(CNN) -- Usain Bolt rounded off the world championships Sunday by claiming his third gold in Moscow as he anchored Jamaica to victory in the men's 4x100m relay.
The fastest man in the world charged clear of United States rival Justin Gatlin as the Jamaican quartet of Nesta Carter, Kemar Bailey-Cole, Nickel Ashmeade and Bolt won in 37.36 seconds.
The U.S finished second in 37.56 seconds with Canada taking the bronze after Britain were disqualified for a faulty handover.
The 26-year-old Bolt has now collected eight gold medals at world championships, equaling the record held by American trio Carl Lewis, Michael Johnson and Allyson Felix, not to mention the small matter of six Olympic titles.
The relay triumph followed individual successes in the 100 and 200 meters in the Russian capital.
"I'm proud of myself and I'll continue to work to dominate for as long as possible," Bolt said, having previously expressed his intention to carry on until the 2016 Rio Olympics.
Victory wa

But which of these summaries comes closest to the reference summary? To this end, we evaluate the results with the ROUGE metric set, which measures the overlap of n-grams between the machine-generated and reference summaries.

- ROUGE-1 measures the overlap of unigrams (single words) between the generated and reference summary , implying the generated summary captures more key terms from the reference.

- ROUGE-2 assesses the overlap of bigrams (two-word sequences), suggesting more phrase-level fidelity and detail in the generated summary relative to the reference.

- ROUGE-L considers the longest common subsequence, highlighting fluency and coherence.

- ROUGE-Lsum is a variant of ROUGE-L specifically tuned for summarization tasks, assessing sentence splits, a higher score suggests a better overall structural and sentence-level match to the reference summary.


In [ ]:
rouge_metric = load_metric("rouge")  # Load the ROUGE metric

In [ ]:
reference = dataset["train"][1]["highlights"]  # Get the reference summary from the dataset
records = []  # Initialize an empty list to store ROUGE scores
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]  # Define the names of the ROUGE metrics

for model_name in summaries:  # Iterate over the generated summaries
    rouge_metric.add(prediction=summaries[model_name], reference=reference)  # Add the prediction and reference to the ROUGE metric
    score = rouge_metric.compute()  # Compute the ROUGE scores
    rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)  # Create a dictionary of ROUGE scores
    records.append(rouge_dict)  # Append the ROUGE scores to the list

pd.DataFrame.from_records(records, index=summaries.keys())  # Create a DataFrame from the ROUGE scores


,rouge1,rouge2,rougeL,rougeLsum
baseline,0.303571,0.090909,0.214286,0.232143
gpt2,0.212121,0.000000,0.121212,0.212121
DeepSeek,0.195402,0.046512,0.149425,0.172414
bart,0.582278,0.207792,0.455696,0.506329
sumy,0.218579,0.055249,0.131148,0.185792


Looking at the results we can make several conclusions: 

1. Baseline (Three-Sentence Summary): ROUGE-1: 0.30 and ROUGE-L: 0.21 indicate that while the baseline method captures some key words from the reference summary, it lacks depth in fluency (ROUGE-L shows limited coherence). ROUGE-2: 0.09 is relatively low, suggesting a lack of capturing phrase-level information.

2. GPT-2: ROUGE-1: 0.21 shows reduced unigram overlap compared to the baseline, which might mean GPT-2 generated broader text rather than sticking to precision words. ROUGE-2: 0.00 indicates GPT-2 did not capture any bigram matches, suggesting generated summaries missed critical phrase-level consistency. ROUGE-L: 0.12 suggests lesser fluency and coherence than the baseline.

3. DeepSeek: ROUGE-1: 0.19 and ROUGE-2: 0.04, both lower than the baseline, showing minimal key content and phrase overlap.

4. BART: ROUGE-1: 0.58 and ROUGE-2: 0.20 are significantly higher than other models, highlighting BART’s effectiveness in retaining both key terms and phrase-level details. ROUGE-L: 0.45 and ROUGE-Lsum: 0.50 demonstrate strong fluency, coherence, and effectiveness in summarizing content akin to the reference.

5. Sumy (TextRank): ROUGE-1: 0.21 and ROUGE-2: 0.05 slightly outperform GPT-2 and DeepSeek, indicating better capture of key terms and some phrases. ROUGE-L: 0.13 suggests moderate coherence.

BART performs best across all ROUGE metrics, which is not surprising given that it has been pre-trained specifically on the CNN/DailyMail dataset. This pre-training allows BART to generate summaries with extensive content overlap and coherence when compared to the reference summaries. However, its high performance should be viewed in the context of its familiarity with the dataset, suggesting that BART may be particularly well-suited for tasks involving similar data but may not generalize as well to entirely new datasets without further fine-tuning. Furthermore we see that GPT-2 and (a slim version of) DeepSeek perform comparably, while the classical method is performing in a comparable range as LLM models. 

In summary, the performance analysis of different summarization approaches highlights the importance of model-data alignment in achieving effective results. BART’s remarkable performance underscores the advantages of using models fine-tuned on specific datasets, though with the caveat of potentially limited generalizability. Meanwhile, GPT-2 and DeepSeek illustrate the capabilities of general LLMs, and the classical TextRank approach remains a viable option, especially in contexts lacking expansive computational resources or annotated data. As you consider implementing these techniques, balance model selection with the specific needs and constraints of your intended application, and be mindful of the potential need for fine-tuning to optimize performance across diverse datasets.
